In [1]:
import os
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm  # Import tqdm for the progress bar
from evaluator import evaluate
from eslint import EslintProcessor
from google_closure_compiler import GoogleClosureCompiler
from chatgpt_api import ChatGPT  # Import ChatGPT
from gemini_api import gemini_response,gemini_json_response # Import Gemini API
from collections import defaultdict
import enum
from typing_extensions import TypedDict
from typing import List, Optional
from pydantic import BaseModel
from groq import Groq

In [38]:
code_gen = pd.read_csv("Code_Generation_Dataset_with_Linter_Outputs.csv")
code_gen.head()

,Problem Number,Name,Description,Examples,Solution,Constraints,Difficulty,Link,Function Header,Linter Outputs
0,1,Create Hello World Function,Write a function createHelloWorld. It should r...,"Example 1:\n\nInput: args = []\nOutput: ""Hello...",var createHelloWorld = function() { \n retu...,0 <= args.length <= 10,Easy,https://leetcode.com/problems/create-hello-wor...,/**\n * @return {Function}\n */\nvar createHel...,"ESLint Output:\n\n[{'ruleId': 'no-var', 'sever..."
1,2,Counter 2,Write a function createCounter. It should acce...,"Example 1:\n\nInput: init = 5, calls = [""incre...",var createCounter = function(init) {\n let pr...,-1000 <= init <= 1000\n0 <= calls.length <= 10...,Easy,https://leetcode.com/problems/counter-ii/descr...,/**\n * @param {integer} init\n * @return { in...,"ESLint Output:\n\n[{'ruleId': 'no-var', 'sever..."
2,3,Apply Transform Over Each Element in Array,Given an integer array arr and a mapping funct...,"Example 1:\n\nInput: arr = [1,2,3], fn = funct...","var map = function(arr, fn) {\r\n const tra...",0 <= arr.length <= 1000\n-10^9 <= arr[i] <= 10...,Easy,https://leetcode.com/problems/apply-transform-...,/**\n * @param {number[]} arr\n * @param {Func...,"ESLint Output:\n\n[{'ruleId': 'no-var', 'sever..."
3,4,Filter Elements from Array,Given an integer array arr and a filtering fun...,"Example 1:\n\nInput: arr = [0,10,20,30], fn = ...","var filter = function(arr, fn) {\n const resu...",0 <= arr.length <= 1000\n-10^9 <= arr[i] <= 10^9,Easy,https://leetcode.com/problems/filter-elements-...,/**\n * @param {number[]} arr\n * @param {Func...,"ESLint Output:\n\n[{'ruleId': 'no-var', 'sever..."
4,5,Array Reduce Transformation,"Given an integer array nums, a reducer functio...","Example 1:\n\nInput: \nnums = [1,2,3,4]\nfn = ...",/**\n * @param {number[]} nums\n * @param {Fun...,0 <= nums.length <= 1000\n0 <= nums[i] <= 1000...,Easy,https://leetcode.com/problems/array-reduce-tra...,/**\n * @param {number[]} nums\n * @param {Fun...,"ESLint Output:\n\n[{'ruleId': 'no-var', 'sever..."


In [18]:
PROMPT_CODEGEN_V1="""
Task: Generate JavaScript code based on the following description.

Problem Description: {leetcode_desc}

Give your output in this particular schema: {schema}

Requirements:
1. Use modern JavaScript syntax (ES6+)
2. Implement error handling where appropriate
3. Follow best practices for code organization and readability
"""

PROMPT_REFACTOR_V1="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

PROMPT_REFACTOR_V2="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

out_schema = {
    "code": "generated javascript code",
    "rationale": "explain your approach keep it under 200 words"
}


# Experiment 3: Refactor with metadata from LeetCode added
PROMPT_REFACTOR_V3="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Examples: {examples}

Constraint: {constraint}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
"""

# Experiment 4: Refactor with linter output given in the prompt. (need linter output from Jacob)
PROMPT_REFACTOR_V4="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm
2. Improve the readability and quality of algorithm
3. Use the suggestions provided the Linter to optimize the code

Linter output of the original code: {linter_output}
"""

# Experiment 5: Refactor with more specific requirements (e.g., make the solution 10 lines or less) (not trivial, the above cases first)
PROMPT_REFACTOR_V5="""
Task: Refactor the following JavaScript code to improve its quality, readability, and maintainability.

Problem Description: {leetcode_desc}

Original Code: {gt_code}

Examples: {examples}

Constraint: {constraint}

Give your output in this particular schema: {schema}

Requirements:
1. Improve the run-time of the algorithm if possible
2. Improve the readability and quality of algorithm
3. Algorithm Optimization: Suggest a more efficient algorithm in terms of run-time if possible. Consider using appropriate data structures like hash tables for O(1) lookups or binary search trees for O(log n) operations 
4. Code Optimization: Minimize unnecessary loops and nested loops. Use efficient built-in functions and methods specific to the programming language. Reduce redundant calculations by caching results or using dynamic programming
5. Memory Optimization: Optimize memory usage by using appropriate data types. Consider in-place algorithms to reduce space complexity.
"""

In [4]:
groq = Groq(api_key='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')

In [19]:
# Data model for LLM to generate
# class Code_(BaseModel):
#     name: str
#     quantity: str
#     quantity_unit: Optional[str]


class Code_Schema(BaseModel):
    code: str
    rationale: str


def gen_code(code_prompt: str,model_name: str):
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a code generator that outputs code and its rationale in JSON. Generate code in Javascript and explain your approach keep it under 200 words in rationale.\n"
                f" The JSON object must use the schema: {json.dumps(out_schema, indent=2)}",
#                 f" The JSON object must use the schema: {json.dumps(Code_Schema.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"{code_prompt}",
            },
        ],
        model= model_name,
        temperature=0,
        stream=False,
        response_format={"type": "json_object"},
    )
    return chat_completion.choices[0].message.content


# def print_recipe(recipe: Recipe):
#     print("Recipe:", recipe.recipe_name)

#     print("\nIngredients:")
#     for ingredient in recipe.ingredients:
#         print(
#             f"- {ingredient.name}: {ingredient.quantity} {ingredient.quantity_unit or ''}"
#         )
#     print("\nDirections:")
#     for step, direction in enumerate(recipe.directions, start=1):
#         print(f"{step}. {direction}")


# recipe = get_recipe("apple pie")
# print_recipe(recipe)


In [39]:
model_name = "mixtral-8x7b-32768"
MAX_TOKENS_ANSWER = 1000
ans = []
rat = []
err_lst = []
for index, row in code_gen.iterrows():
    try:
        code = row['Solution']
        desc = row['Description']
        ex = row['Examples']
        prob = row['Constraints']
        schema = row['Function Header']
        lint = row['Linter Outputs']
        code_prompt = PROMPT_REFACTOR_V4.format(schema=schema, gt_code = code, leetcode_desc = desc, constraint = prob, examples=ex, linter_output= lint)
        response_txt = gen_code(code_prompt, model_name)
#             print(response_txt)
        response_json = json.loads(response_txt)
        ans.append(response_json['code'])
        rat.append(response_json['rationale'])
        print(index)
        print("-------")
    except Exception as e:
        error_msg = f"An error occurred: {str(e)}"
        print(error_msg)
        print(f"failed_{index}")
        print(response_txt)
        ans.append(index)
        rat.append(index)
        err_lst.append(index)


0
-------
1
-------
2
-------
3
-------
4
-------
5
-------
6
-------
7
-------
8
-------
An error occurred: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '{\n  "code": "class EventEmitter {\\n  \\n    /**\\n    * @param {string} eventName\\n    * @param {Function} callback\\n    * @return {Object}}'}}
failed_9
{
  "code": "/**\n * @param {!Array<!Function>} functions\n * @return {!Promise<!Array>}\n */\nvar promiseAll = function(functions) {\n  if (functions.length === 0) {\n    return Promise.resolve([]);\n  }\n\n  const results = new Array(functions.length);\n  let resolvedCount = 0;\n\n  return Promise.all(functions.map(fn => {\n    return fn().then(val => {\n      results[resolvedCount] = val;\n      resolvedCount++;\n\n      if (resolvedCount === functions.length) {\n        return results;\n      }\n    }

46
-------
47
-------
48
-------
49
-------


In [41]:
# ans[11]= "class Calculator {\n  /**\n   * @param {number} value\n   */\n  constructor(value) {\n    this.result = value || 0;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  add(value) {\n    this.result += value || 0;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  subtract(value) {\n    this.result -= value || 0;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  multiply(value) {\n    this.result *= value || 1;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  divide(value) {\n    if (value === 0) {\n      throw new Error('Division by zero is not allowed');\n    }\n    this.result /= value;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  power(value) {\n    if (value === 0) {\n      this.result = 1;\n    } else {\n      this.result = Math.pow(this.result, value);\n    }\n    return this;\n  }\n\n  /**\n   * @return {number}\n   */\n  getResult() {\n    return this.result;\n  }\n}"
# rat[11] = "The refactored code has several improvements:\n- Default values are assigned to the parameters to prevent NaN results when adding or subtracting from zero.\n- The multiplication operation has a default value of 1 for the parameter to avoid multiplying by zero and returning zero.\n- The power operation checks if the value is zero and directly assigns 1 to the result, avoiding the expensive call to Math.pow.\n- JSDoc comments are added to improve readability and provide clear documentation for each method."
# ans[41] = "var maxDepth = function(root) {\n    if (!root) return 0;\n    return Math.max(maxDepth(root.left), maxDepth(root.right)) + 1;\n};"
# rat[41] = "The original code is already efficient as it uses recursion to calculate the maximum depth of the binary tree. However, it can be improved in terms of readability. The proposed solution removes the leftSubHeight and rightSubHeight variables and directly compares the maximum depth of the left and right subtrees. This makes the code more concise and easier to understand. The time complexity remains the same, O(n), where n is the number of nodes in the tree, because each node is visited only once."
# rat[47] = "/**\n * @param {string} s\n * @param {string} p\n * @return {boolean}\n */\nvar isMatch = function(s, p) {\n  const dp = new Array(s.length + 1).fill(0).map(() => new Array(p.length + 1).fill(false));\n  dp[0][0] = true;\n  for (let j = 1; j < p.length + 1; j++) {\n    if (p[j - 1] === '*') {\n      dp[0][j] = dp[0][j - 2];\n    }\n  }\n  for (let i = 1; i < s.length + 1; i++) {\n    for (let j = 1; j < p.length + 1; j++) {\n      if (s[i - 1] === p[j - 1] || p[j - 1] === '?') {\n        dp[i][j] = dp[i - 1][j - 1];\n      } else if (p[j - 1] === '*') {\n        dp[i][j] = dp[i][j - 2] || dp[i - 1][j];\n      }\n    }\n  }\n  return dp[s.length][p.length];\n};"
# rat[47] = "The original code uses a single loop to iterate over the input string and a separate pointer to keep track of the pattern. This approach can be improved by using a 2D boolean array (dp) to store the results of previously computed subproblems, reducing the time complexity from O(2^n) to O(n^2). The code is made more readable by adding comments and using descriptive variable names. The space complexity of the algorithm is increased due to the use of the dp array, but this trade-off is acceptable for the improved time complexity."

# ans[24] = "/**\n * @param {number[][]} isConnected\n * @return {number}\n */\nvar findCircleNum = function(isConnected) {\n   let provinces = 0;\n   let visited = new Set();\n   for (let i = 0; i < isConnected.length; i++) {\n      if (dfs(isConnected, visited, i)) {\n         provinces++;\n      }\n   }\n   return provinces;\n};\n\nfunction dfs(isConnected, visited, i) {\n   if (visited.has(i)) {\n      return false;\n   }\n   visited.add(i);\n   for (let j = 0; j < isConnected[i].length; j++) {\n      if (isConnected[i][j] === 1) {\n         dfs(isConnected, visited, j);\n      }\n   }\n   return true;\n}"
# rat[24] = "The original code uses an array to keep track of visited nodes, which results in a time complexity of O(n^2) due to the nested loops. This can be improved by using a Set to keep track of visited nodes, reducing the time complexity to O(n) for the dfs function. Additionally, the code readability is improved by adding JSDoc comments, using descriptive variable names, and formatting the code consistently."

# ans[11] = "class Calculator {\n  /**\n   * @param {number} value\n   */\n  constructor(value) {\n    this.result = value || 0;\n  }\n   \n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  add(value) {\n    this.result += (value || 0);\n    return this;\n  }\n   \n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  subtract(value) {\n    this.result -= (value || 0);\n    return this;\n  }\n   \n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  multiply(value) {\n    this.result *= (value || 1);\n    return this;\n  }\n   \n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  divide(value) {\n    if (value === 0) {\n      throw new Error('Division by zero is not allowed');\n    }\n    this.result /= (value || 1);\n    return this;\n  }\n   \n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  power(value) {\n    if (value === 0) {\n      this.result = 1;\n    } else {\n      this.result = Math.pow(this.result, value);\n    }\n    return this;\n  }\n   \n  /**\n   * @return {number}\n   */\n  getResult() {\n    return this.result;\n  }\n}"
# rat[11] = "The refactored code improves readability and maintainability by adding JSDoc comments for each method, providing a clear description of the input parameters and return values. Additionally, default values are provided for the add, subtract, multiply, and power methods to prevent unexpected behavior when a value of 0 or undefined is passed. The divide method was updated to allow division by 1 as a default value, which is mathematically equivalent to the original operation. The power method was updated to handle the case where the exponent is 0, setting the result to 1 in this case. This avoids the need for an external check of the result after performing a power operation with an exponent of 0."
# ans[16] = "var productExceptSelf = function(nums) {\n    const output = [1];\n    let left = 1;\n    for (let i = 0; i < nums.length; i++) {\n        output.push(left);\n        left *= nums[i];\n    }\n    let right = 1;\n    for (let i = nums.length - 1; i >= 0; i--) {\n        output[i] *= right;\n        right *= nums[i];\n    }\n    return output;\n};\n"
# rat[16] = "The original code has a time complexity of O(n) which is optimal for this problem. However, it can be improved in terms of readability and quality.\n\nIn the original code, the output array is initialized with n 1's, where n is the length of the input array. This can be simplified by initializing the output array with a single 1, and then pushing the left product to the output array in each iteration of the first loop. This reduces the number of array accesses and makes the code more readable.\n\nFurthermore, the variable names can be made more descriptive. Instead of 'output', 'left', and 'right', we can use 'productExceptSelf', 'prefixProduct', and 'suffixProduct' respectively, to make the code more self-explanatory."
# ans[21] = "/**\n * @param {string} senate\n * @return {string}\n */\nvar predictPartyVictory = function(senate) {\n  const n = senate.length;\n  const senators = new Array(n).fill(0).map((_, i) => i % 2 === 0 ? 'R' : 'D');\n  let radiantBanned = 0;\n  let direBanned = 0;\n\n  for (let i = 0; i < n; i++) {\n    if (senators[i] === 'R') {\n      if (direBanned === n - radiantBanned - 1) {\n        return 'Radiant';\n      }\n      direBanned++;\n    } else {\n      if (radiantBanned === n - direBanned - 1) {\n        return 'Dire';\n      }\n      radiantBanned++;\n    }\n  }\n};"
# rat[21] ="The original code has a time complexity of O(n^2) due to the inner loop. This refactored code improves the run-time of the algorithm by using two variables, radiantBanned and direBanned, to keep track of the number of banned senators from each party. The code iterates through the senate string only once, with a time complexity of O(n), where n is the length of the senate string.\n\nThe readability and quality of the algorithm are also improved by:\n- Adding a docstring to describe the function's input and output.\n- Replacing the string manipulation with a more straightforward array manipulation.\n- Simplifying the conditions and removing the unnecessary break statement."
# ans[33] = "/**\n * @param {number[]} nums\n * @param {number} target\n * @return {number}\n */\nvar search = function(nums, target) {\n  let left = 0;\n  let right = nums.length - 1;\n\n  while (left < right) {\n    const mid = left + Math.floor((right - left) / 2);\n\n    if (nums[mid] === target) {\n      return mid;\n    } else if (nums[mid] < target) {\n      left = mid + 1;\n    } else {\n      right = mid;\n    }\n  }\n\n  return -1;\n};"
# rat[33] = "The original code has a minor issue where the right index is updated to mid - 1 instead of mid when the target is greater than the middle element. This can lead to an incorrect result when the target is at the very end of the array. The updated code fixes this issue by setting right = mid instead of right = mid - 1. Additionally, the code has been slightly reformatted to improve readability, such as using let instead of var, and calculating the middle index as left + (right - left) / 2 to avoid potential integer overflow issues."


# ans[11] = "class Calculator {\n  /**  * @param {number} value */\n  constructor(value) {\n    this.result = value || 0;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  add(value) {\n    this.result += value || 0;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  subtract(value) {\n    this.result -= value || 0;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  multiply(value) {\n    this.result *= value || 1;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  divide(value) {\n    if (value === 0) {\n      throw new Error('Division by zero is not allowed');\n    }\n    this.result /= value;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  power(value) {\n    if (value === 0) {\n      this.result = 1;\n      return this;\n    }\n    this.result = Math.pow(this.result, value);\n    return this;\n  }\n   \n  /**  * @return {number} */\n  getResult() {\n    return this.result;\n  }\n}\n"
# rat[11] = "The refactored code has the following improvements:\n- Added default values for method parameters to prevent errors when calling methods without arguments.\n- Moved the initial value of result to the constructor's default parameter to allow creating a Calculator instance without an argument.\n- Changed the divisor's default value in the divide method to 1 to avoid changing the result when dividing by 0.\n- Added a check for the power method's parameter to return 1 when raising to the power of 0, avoiding unnecessary calculations.\n- Kept the same time and space complexity as the original code: O(1) for all methods."
# ans[12] = "class Calculator {\n  /**  * @param {number} value */\n  constructor(value) {\n    this.result = value || 0;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  add(value) {\n    this.result += value || 0;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  subtract(value) {\n    this.result -= value || 0;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  multiply(value) {\n    this.result *= value || 1;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  divide(value) {\n    if (value === 0) {\n      throw new Error('Division by zero is not allowed');\n    }\n    this.result /= value;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  power(value) {\n    if (value === 0) {\n      this.result = 1;\n      return this;\n    }\n    this.result = Math.pow(this.result, value);\n    return this;\n  }\n   \n  /**  * @return {number} */\n  getResult() {\n    return this.result;\n  }\n}\n"
# rat[12] = "The refactored code has the following improvements:\n- Added default values for method parameters to prevent errors when calling methods without arguments.\n- Moved the initial value of result to the constructor's default parameter to allow creating a Calculator instance without an argument.\n- Changed the divisor's default value in the divide method to 1 to avoid changing the result when dividing by 0.\n- Added a check for the power method's parameter to return 1 when raising to the power of 0, avoiding unnecessary calculations.\n- Kept the same time and space complexity as the original code: O(1) for all methods."
# ans[13] = "class Calculator {\n  /**  * @param {number} value */\n  constructor(value) {\n    this.result = value || 0;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  add(value) {\n    this.result += value || 0;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  subtract(value) {\n    this.result -= value || 0;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  multiply(value) {\n    this.result *= value || 1;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  divide(value) {\n    if (value === 0) {\n      throw new Error('Division by zero is not allowed');\n    }\n    this.result /= value;\n    return this;\n  }\n   \n  /**  * @param {number} value * @return {Calculator} */\n  power(value) {\n    if (value === 0) {\n      this.result = 1;\n      return this;\n    }\n    this.result = Math.pow(this.result, value);\n    return this;\n  }\n   \n  /**  * @return {number} */\n  getResult() {\n    return this.result;\n  }\n}\n"
# rat[13] = "The refactored code has the following improvements:\n- Added default values for method parameters to prevent errors when calling methods without arguments.\n- Moved the initial value of result to the constructor's default parameter to allow creating a Calculator instance without an argument.\n- Changed the divisor's default value in the divide method to 1 to avoid changing the result when dividing by 0.\n- Added a check for the power method's parameter to return 1 when raising to the power of 0, avoiding unnecessary calculations.\n- Kept the same time and space complexity as the original code: O(1) for all methods."
# ans[19] = "/**\n * @param {number[]} nums1\n * @param {number[]} nums2\n * @return {number[][]}\n */\nvar findDifference = function(nums1, nums2) {\n  const set1 = new Set(nums1);\n  const set2 = new Set(nums2);\n  const result = [[], []];\n\n  for (const num of nums1) {\n    if (!set2.has(num)) {\n      result[0].push(num);\n    }\n  }\n\n  for (const num of nums2) {\n    if (!set1.has(num)) {\n      result[1].push(num);\n    }\n  }\n\n  return result;\n};\n"
# rat[19] = "The original code is already efficient, with a time complexity of O(n + m) where n and m are the lengths of nums1 and nums2 respectively. However, it can be improved for readability and maintainability.\n\n1. The code is made more readable by initializing the result array to [[], []] and pushing distinct elements to their respective arrays in the result.\n2. This approach avoids creating new arrays using Array.from() and filter(), thus reducing memory usage.\n3. The time complexity remains O(n + m) as we are only iterating through the arrays once.\n4. This refactoring improves code readability and maintainability without compromising performance."
# ans[21] = "/**\n * @param {string} senate\n * @return {string} \n*/\nvar predictPartyVictory = function(senate) {\n  let len = senate.length;\n  let index = 0;\n  let banned = new Set();\n  while(len-- > 0) {\n    let senator = senate[index];\n    index = (index + 1) % len;\n    if(banned.has(index)) continue;\n    banned.add(index);\n    let start = index;\n    while(senate[index] === senator) {\n      index = (index + 1) % len;\n      if(index === start) break;\n      if(!banned.has(index)) banned.add(index);\n    }\n    if(index === start) return senator === 'D' ? 'Dire' : 'Radiant';\n  }\n};"
# rat[21] = "The original code has a time complexity of O(n^2) due to the nested loop. This refactored code improves the time complexity to O(n) by using a single loop and a Set to keep track of banned senators. The Set data structure allows for O(1) lookups, which further optimizes the performance. Additionally, the modulo operator is used to loop back to the beginning of the senate string, improving readability and reducing redundant calculations."
# ans[49] = "/**\n * @param {number[]} nums\n * @return {number}\n */\nvar maxCoins = function(nums) {\n    const n = nums.length + 2;\n    const paddedNums = new Array(n).fill(1).map((v, i) => i < nums.length ? nums[i] : 1);\n    let dp = new Array(n).fill(0).map(() => new Array(n).fill(0));\n\n    for (let len = 2; len < n; len++) {\n        for (let left = 1; left + len - 1 < n; left++) {\n            let right = left + len - 1;\n            dp[left][right] = Math.max(...Array.from({\n                length: len\n            }, (_, idx)\n                => paddedNums[left + idx - 1] * dp[left][left + idx - 1] * paddedNums[right - idx] + (idx < len - 1 ? dp[left + idx][right] : 0)))\n        }\n    }\n\n    return dp[1][n - 1];\n};\n"
# rat[49] = "The original code has a time complexity of O(n^3) and space complexity of O(n^2). The code can be refactored to reduce the time complexity by using a more efficient approach to calculate the maximum coins. The refactored code uses a dynamic programming approach with a time complexity of O(n^3) and space complexity of O(n^2). However, the space complexity can be further optimized by reusing the same 2D array for calculating the maximum coins for all subarrays of different lengths. The refactored code initializes a 2D array of size (n+2) x (n+2) and reuses it for calculating the maximum coins for all subarrays of different lengths. This reduces the space complexity to O(n^2). The code is also made more readable by using array comprehensions and destructuring assignments."

ans[9] = "/**\n * @param {!Array<!Function>} functions\n * @return {!Promise<!Array>}\n */\nvar promiseAll = function(functions) {\n  if (functions.length === 0) {\n    return Promise.resolve([]);\n  }\n\n  const results = new Array(functions.length);\n  let resolvedCount = 0;\n\n  return Promise.all(functions.map(fn => {\n    return fn().then(val => {\n      results[resolvedCount] = val;\n      resolvedCount++;\n\n      if (resolvedCount === functions.length) {\n        return results;\n      }\n    });\n  }));\n};"
rat[9] = "The refactored code improves the run-time of the algorithm by using Promise.all, which resolves when all promises are resolved or the first one is rejected. It also improves the readability and quality of the algorithm by using const and let instead of var, and by adding JSDoc comments for better understanding of the function's parameters and return value. The code also handles the case when the array of functions is empty, by returning a resolved promise with an empty array. The use of Promise.all reduces the complexity of the code and makes it more maintainable."
ans[11] = "class Calculator {\n  /**\n   * @param {number} value\n   */\n  constructor(value) {\n    this.result = value;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  add(value) {\n    this.result += value;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  subtract(value) {\n    this.result -= value;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  multiply(value) {\n    this.result *= value;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  divide(value) {\n    if (value === 0) {\n      throw new Error('Division by zero is not allowed');\n    }\n    this.result /= value;\n    return this;\n  }\n\n  /**\n   * @param {number} value\n   * @return {Calculator}\n   */\n  power(value) {\n    this.result **= value;\n    return this;\n  }\n\n  /**\n   * @return {number}\n   */\n  getResult() {\n    return this.result;\n  }\n}"
rat[11] = "The code was refactored to improve readability and maintainability. JSDoc comments were added to describe the constructor and methods, as suggested by the linter. The 'Math.pow' method was replaced with the exponentiation operator '**', as suggested by the linter. The class was left unchanged, as it was already well-structured and efficient."
ans[34] = "/**\n * @param {!number[]} nums\n * @export\n */\nvar moveZeroes = function(nums) {\n  let start = 0;\n  for (let end = 0; end < nums.length; end++) {\n    if (nums[end] !== 0) {\n      [nums[start], nums[end]] = [nums[end], nums[start]];\n      start++;\n    }\n  }\n};"
rat[34] = "The refactored code improves run-time, readability, and maintainability by using let instead of var, removing unnecessary comments, and using destructuring assignment. The run-time is improved by starting the inner loop from the current position of the non-zero element instead of 0, reducing the number of iterations. The code is made more readable by adding JSDoc comments, which provide clear input and output descriptions. Destructuring assignment is used to swap elements, making the code more concise and easier to understand."
ans[45] = "/**\n * @param {string} s\n * @return {string}\n */\nvar shortestPalindrome = function(s) {\n    const pi = kmp(s.split('').reverse().join(''), s);\n    return s.slice(pi.length).split('').reverse().join('') + s;\n};\n\n/**\n * @param {string} txt\n * @param {string} patt\n * @return {!Array<number>}\n */\nconst kmp = (txt, patt) => {\n    const newString = `${patt}#${txt}`;\n    const pi = new Array(newString.length).fill(0);\n    let i = 1, k = 0;\n    while (i < newString.length) {\n        if (newString[i] === newString[k]) {\n            k++;\n            pi[i] = k;\n            i++;\n        } else {\n            if (k > 0) {\n                k = pi[k - 1];\n            } else {\n                pi[i] = 0;\n                i++;\n            }\n        }\n    }\n    return pi;\n};"
rat[45] = "The code was refactored to improve readability, quality, and maintainability. The variable declarations were changed from 'var' to 'const' since their values do not change. The function declarations were changed to arrow functions for brevity. The string concatenation was changed to template literals for readability. The while loop was refactored to eliminate the use of the '++' operator. The Linter suggestions were implemented to improve the code quality. The Google Closure Compiler output was also addressed by adding JSDoc comments to the functions."

In [40]:
err_lst

[9, 11, 34, 45]

In [42]:
output_df = pd.DataFrame()
output_df['gt_code'] = code_gen['Solution']
output_df['level'] = code_gen['Difficulty']
output_df['refactor_code'] = ans
output_df['rationale'] = rat

output_df

,gt_code,level,refactor_code,rationale
0,var createHelloWorld = function() { \n retu...,Easy,/**\n * @return {Function} \n */\nlet createHe...,The given code can be refactored using ES6 arr...
1,var createCounter = function(init) {\n let pr...,Easy,/**\n * @param {number} init\n * @return {{inc...,The code was refactored to improve readability...
2,"var map = function(arr, fn) {\r\n const tra...",Easy,/**\n * @param {!number[]} arr\n * @param {!fu...,The given code was already quite clean and eff...
3,"var filter = function(arr, fn) {\n const resu...",Easy,/**\n * @param {!number[]} arr\n * @param {fun...,The code was refactored to improve its quality...
4,/**\n * @param {number[]} nums\n * @param {Fun...,Easy,/**\n * @param {!Array<number>} nums\n * @para...,The refactored code improves the run-time of t...
5,/**\n * @param {Function} fn\n * @param {numbe...,Medium,/**\n * @param {!Function} fn\n * @param {numb...,The code was refactored to improve its quality...
6,const TimeLimitedCache = function() {\n thi...,Medium,var TimeLimitedCache = function() {\n this....,The code was refactored to improve readability...
7,"var debounce = function(fn, t) {\n let id;\...",Medium,/**\n * @param {Function} fn\n * @param {numbe...,The code was refactored to use ES6 arrow funct...
8,var promiseAll = async function(functions) {\n...,Medium,/**\n * @param {!Array<!Function>} functions\n...,The refactored code improves the run-time of t...
9,class EventEmitter {\n constructor() {\n t...,Medium,/**\n * @param {!Array<!Function>} functions\n...,The refactored code improves the run-time of t...


In [43]:
output_df.to_csv(f"output_files/mixtral_results_temp_0/PROMPT_REFACTOR_V4_{model_name}.csv")